In [23]:
RAW_FILE_PATH = '../../data/raw/Transaction.csv'
OUTPUT_PATH = '../../data/'

In [24]:
%run '../../schema_definition/schema.py'

In [25]:
import sys  
sys.path.insert(0, '../../')

from schema_definition.schema import *

In [26]:
from pyspark.shell import spark
from pyspark.sql.types import TimestampType
from pyspark.sql import functions as F

21/10/15 22:04:44 WARN Utils: Your hostname, Arturos-Mac-mini.local resolves to a loopback address: 127.0.0.1; using 192.168.1.26 instead (on interface en1)
21/10/15 22:04:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/10/15 22:04:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.1.2
      /_/

Using Python version 3.8.9 (default, Aug 21 2021 15:53:23)
Spark context Web UI available at http://192.168.1.26:4040
Spark context available as 'sc' (master = local[*], app id = local-1634295885029).
SparkSession available as 'spark'.


In [28]:
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")

DataFrame[key: string, value: string]

In [29]:
raw_df = spark.read.csv(RAW_FILE_PATH, sep=',', header=True, schema=bronze_schema, enforceSchema=True)
raw_df.repartition(1).write.csv(OUTPUT_PATH + 'bronze', header=True, sep='|', mode='overwrite')

21/10/15 22:04:54 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 19, schema size: 17
CSV file: file:///Users/arturogonzalez/DataspellProjects/transactions_notebooks/data/raw/Transaction.csv


In [30]:
bronze_df = spark.read.csv(OUTPUT_PATH + 'bronze/*.csv', sep='|', header=True, schema=silver_schema, enforceSchema=True)

augment = bronze_df.withColumn("HashKey", F.sha2(F.concat_ws("||", *bronze_df.columns), 256))

convert_to_timestamp = augment \
    .withColumn("ImplementedDate", F.unix_timestamp("ImplementedDate", "d/MM/yyyy HH:mm").cast(TimestampType())) \
    .withColumn("RequestDate", F.to_timestamp("RequestDate", 'dd/MM/yyyy HH:mm').cast(TimestampType())) \
    .withColumn("LastUpdatedDate", F.to_timestamp("LastUpdatedDate", 'dd/MM/yyyy HH:mm').cast(TimestampType()))


calculate_response = convert_to_timestamp.withColumn("Response", F.datediff(F.col("RequestDate"), F.col("ImplementedDate")))
calculate_response.createOrReplaceTempView("fastest_response")

fastest_response_query = """
                        SELECT * 
                        FROM fastest_response 
                        ORDER BY Response DESC
                        """
fastest_response_df = spark.sql(fastest_response_query)

drop_nulls = fastest_response_df.na.drop(subset=["AccountID"])
filter_out = drop_nulls.filter(~drop_nulls.Fibre.startswith('2.67E'))
filter_out.repartition(1).write.csv(OUTPUT_PATH + 'silver', sep='|', header=True, mode='overwrite')

21/10/15 22:05:35 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 17, schema size: 19
CSV file: file:///Users/arturogonzalez/DataspellProjects/transactions_notebooks/data/bronze/part-00000-e1272132-8e2d-4918-915d-cdf1595c2ce6-c000.csv
21/10/15 22:05:36 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 17, schema size: 19
CSV file: file:///Users/arturogonzalez/DataspellProjects/transactions_notebooks/data/bronze/part-00000-e1272132-8e2d-4918-915d-cdf1595c2ce6-c000.csv


In [31]:
silver_df = spark.read.csv(OUTPUT_PATH + 'silver/*.csv', sep='|', header=True, schema=gold_schema, enforceSchema=True)
silver_df.repartition(1).write.option("maxRecordsPerFile", 1000).json(OUTPUT_PATH + 'gold', mode='overwrite')